In [24]:
import requests
import pandas as pd

def get_met():

    # Client ID to access data from 
    client_id = 'd933f861-70f3-4d0f-adc6-b1edb5978a9e'

    # Define endpoints and parameters
    endpoint = 'https://frost.met.no/observations/v0.jsonld'
    parameters = {
        'sources': 'SN68860',  # Station ID for Voll weather station
        'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)',  # Requestion temperautre, precipitation, and wind speed
        'referencetime': '2024-02-19/2024-03-19',  # Limiting the time frame for the data request
        'timeresolutions': 'P1D',  # Set to 1 day resolution
    }

    # Send an HTTP GET-request
    r = requests.get(endpoint, params=parameters, auth=(client_id, ''))

    # Extract JSON-data
    json_data = r.json()

    # Check if the request was succesfull, and exit if not
    if r.status_code == 200:
        data = json_data['data']
        print('Data collected from frost.met.no! \n')
    else:
        print('Error! Statuscode:', r.status_code)
        print('Message:', json_data['error']['message'])
        print('Cause:', json_data['error']['reason'])
        exit()  

    # Create and set up the dataframe
    df = pd.DataFrame()
    for obs in data:
        row = pd.DataFrame(obs['observations'])
        row['referenceTime'] = obs['referenceTime']
        df = pd.concat([df, row], ignore_index=True)

    # Remove uneeded collumns
    columns_to_drop = ["level", "timeOffset", "timeResolution", "timeSeriesId", "elementId", "performanceCategory", "exposureCategory", "qualityCode"]
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    # Remove time portion from 'referenceTime' (keep only 'YYYY-MM-DD')
    df["referenceTime"] = df["referenceTime"].str.split("T").str[0]

    #Returns dataframe upon request
    return(df)

# Make pandas show the entire dataframe in a readable manner
def pretty_weather_data():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    display(df)


# *** EVERYTHING below this line is purely for local testing purposes of this specific file ***

    # Run the function to get the dataframe with the weather data
#df=get_met()

    # Show the first few rows of the dataframe
#print(df.head())

    # Run the function to print the data in a readable manner
#pretty_weather_data()

    # To check the number of rows in the dataset
#print(df.shape[0])